less is more - kaggle santander

# start

return to four models

In [1]:
PRFX='0319_5'
trntmstmp=1584412344
valtmstmp=1584626395
import datetime
[datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (trntmstmp, valtmstmp)]

['2020-03-17 02:32:24', '2020-03-19 13:59:55']

In [2]:
grand_total=1.5e8

# setup

In [3]:
from pathlib import Path
import dask
print('dask.__version__', dask.__version__)
import xgboost as xgb
import dask_xgboost
import dask.dataframe as dd
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc, precision_recall_curve
from dask.distributed import Client
import pickle
import lightgbm as lgb
from tqdm import tqdm

import datetime
def dtnow(): return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

SEED=101
HOME='/data/git/recsys20'
p_in=f'{HOME}/input'
p_out=f'{HOME}/output/{PRFX}'
Path(p_out).mkdir(exist_ok=True, parents=True)

from sklearn.metrics import precision_recall_curve, auc, log_loss

def compute_prauc(pred, gt):
    prec, recall, thresh = precision_recall_curve(gt, pred)
    prauc = auc(recall, prec)
    return prauc

def calculate_ctr(gt):
    positive = len([x for x in gt if x == 1])
    ctr = positive/float(len(gt))
    return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

cols=[
'toks',
'hshtgs',
'twtid',
'media',
'links',
'domns',
'twttyp',
'lang',
'tm',

'u1id',
'u1_fllwer_cnt',
'u1_fllwing_cnt',
'u1_vrfed',
'u1_create_tm',

'u2id',
'u2_follower_cnt',
'u2_following_cnt',
'u2_vrfed',
'u2_create_tm',

'u1_fllw_u2',
'reply_tm',
'retwt_tm',
'retwt_cmmnt_tm',
'like_tm',
]

cols_val = cols[:-4]
cols_tgt_tmstmp=[
    'retwt_tm',
    'reply_tm',
    'like_tm',
    'retwt_cmmnt_tm',
]
cols_tgt=[o.split('_tm')[0] for o in cols_tgt_tmstmp]
tgts             = ['Retweet','Reply','Like','RTwCmnt',]
assert cols_tgt == ['retwt',  'reply','like','retwt_cmmnt',]
ntgts=len(tgts)

dask.__version__ 2.12.0


# prepare data

## get maps from dfval

In [4]:
# dfval=dd.read_csv(
#     f'{p_in}/val_{valtmstmp}.tsv',
#     sep='\x01', header=None, names=cols_val,)
# lang2cnt=dict(dfval.lang.value_counts().compute())
# ttl=sum(lang2cnt.values())
# lang2perc={k:v/ttl for k,v in lang2cnt.items()}
langs=['D3164C7FBCF2565DDF915B1B3AEFB1DC', '22C448FF81263D4BAF2A176145EE9EAD', '06D61DCBBE938971E1EA0C38BD9B5446', 'ECED8A16BE2A5E8871FD55F4842F16B1', 'B9175601E87101A984A50F8A62A1C374', '4DC22C3F31C5C43721E6B5815A595ED6', '167115458A0DBDFF7E9C0C53A83BAC9B', '125C57F4FA6D4E110983FB11B52EFD4E', '022EC308651FACB02794A8147AEE1B78', 'FA3F382BC409C271E3D6EAF8BE4648DD', '9BF3403E0EB7EA8A256DA9019C0B0716', '975B38F44D65EE42A547283787FF5A21', '2996EB2FE8162C076D070A4C8D6532CD', 'FF60A88F53E63000266F8B9149E35AD9', '717293301FE296B0B61950D041485825', '3E16B11B7ADE3A22DDFC4423FBCEAD5D', '3820C29CBCA409A33BADF68852057C4A', '9ECD42BC079C20F156F53CB3B99E600E', '76B8A9C3013AE6414A3E6012413CDC3B', 'AEF22666801F0A5846D853B9CEB2E327', '190BA7DA361BC06BC1D7E824C378064D', '1FFD2FE4297F5E70EBC6C3230D95CB9C', 'A0C7021AD8299ADF0C9EBE326C115F6F', 'D413F5FE5236E5650A46FD983AB39212', '48236EC80FDDDFADE99420ABC9210DDF', '691890251F2B9FF922BE6D3699ABEFD2', '920502FAA080485768AA89BC96A55C47', '0331BF70E606D62D92C96CE9AD71A7CF', '89616CFF8EC8637092F885C7EFF43D74', '06BEAB41D66CCFF329D1ED8BA120A6C2', '60FBA0E834CC59D647C3599AD763FFDF', 'C7A400D9AD489ACF673CF12FBB80AAE5', 'E59EF8BB86A6D815331DDF4C467CE0C7', '4249CE88433AEA3F8DCEECF008B3CB95', '544FA32458C903F1125FE6598300A047', 'CB11E9CF42BD0A1BAD5E27BF3422D99D', '3A85BCEC571C3F5AB1069E4924189177', 'FF7EABB5A382356D54D9C41BA0125E09', 'E7BB61D2A87C1E72DF1C7BC292B86A1C', '69C4A33B9AD29AF883D60BA61CC08702', 'F4FD40A716F1572C9A28E9CAA58BE3A5', '259A6F6DFD672CB1F883CBEC01B99F2D', '6431A618DCF7F4CB7F62A95A39BAB77A', '3121F7240D488F74EEED9312E174B217', '54208B51D44E7D91DC2F3DD02ADEDEC2', 'DBEEFB80F8A314311E2B4BD593E11DFE', 'B6D90127A09AB1229731898AEF9D4D7C', 'F3E1016563360F9434FA986CA86C249C', '2216D01F7B48554E4211021A46816FCF', '1BC639981AE88E09129594B11F894A21', 'C942E369C88CE7C56E69A84D04319FF0', '57ADD4576E2AD6648E9B2DE32F3462A5', 'AC1F0671A4B0D5B8112F87DE7B490E6D', 'C2EF5FABE7619D8A333D5F0FF76E1BFA', 'AA0254541959271ED3453119B787D0C3', 'A6B70CDF8C7B934D4A218CA9B6B7FDB4', 'BF477808A37E3E4E9C5D9F1839E8519E', '97F81BD92A1ACA3F1F43C154E689350F', '60A3DB168094D41241E45E0DE3539BC0', '12D8CEB94F89D11D7EB95EAE9689B009', 'F73266A79468BB89C4325FDEDB0B533C', '4690215948DBF6872B8ED1C2BC87B17E', 'D18801336202297E6484F634CAC6592E', '2E18F6F53E3CF073911AF0A93BBE5373', 'B2235C8B73239FDC5780DD132419833A', '515E873C86EE1577E75FA2387B7FA59E',]

# dfval.twttyp.value_counts().compute()
# TopLevel    2932
# Retweet      994
# Quote        213
twttyps=['TopLevel','Retweet','Quote']

In [5]:
def prp_df(df, istrn=True):
    df['len_toks']=df.toks.apply(len)
    for media in ['Photo', 'Video', 'GIF']:
        df[f'has_media_{media}'] = df.media.fillna('').apply(lambda x: media in x)
    for col in ['hshtgs', 'links', 'domns',]:
        df[f'num_{col}'] = df[col].fillna('').apply(lambda x: len(x.split('\t')) if len(x) else 0)
    tm_dt=pd.to_datetime(df.tm, unit='s')
    df['tm_month']=tm_dt.dt.month
    df['tm_dayofweek']=tm_dt.dt.dayofweek
    df['tm_day']=tm_dt.dt.day
    df['tm_hour']=tm_dt.dt.hour
    df['tmdlta_u2u1']  = df.u2_create_tm - df.u1_create_tm
    df['tmdlta_twtu1'] = df.tm - df.u1_create_tm
    df['tmdlta_twtu2'] = df.tm - df.u2_create_tm

    for typ in twttyps:
        df[f'twttyp_{typ}']=(df.twttyp==typ).astype('int8')

    for lang in langs:
        df[f'lang_{lang}']=(df.lang==lang).astype('int8')
    if istrn: 
        df[cols_tgt]=df[cols_tgt_tmstmp].notna().astype('int8')
        df.drop(inplace=True, columns=['toks', 'hshtgs', 'media', 'links', 'domns', 'twttyp', 'lang', 'twtid', 'u1id', 'u2id']+cols_tgt_tmstmp, )
    else:
        df.drop(inplace=True, columns=['toks', 'hshtgs', 'media', 'links', 'domns', 'twttyp', 'lang', 'u1id',])        
    return df

In [6]:
grand_total/1e6

150.0

In [12]:
chnks_trn = pd.read_csv(f'{p_in}/trn_{trntmstmp}.tsv',sep='\x01',
                    header=None,names=cols, 
                        chunksize=1e4
                       )
for ichnk,df in enumerate(chnks_trn):
    mintm,maxtm=df.tm.min(),df.tm.max()
    mintm,maxtm=(datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (mintm,maxtm))
#     display(df.head(1))
    print(dtnow(), 'chunk', ichnk, mintm,maxtm)
ichnk

2020-03-20 02:50:14 chunk 0 2020-02-06 00:00:02 2020-02-12 23:59:01
2020-03-20 02:50:14 chunk 1 2020-02-06 00:00:03 2020-02-12 23:57:29
2020-03-20 02:50:14 chunk 2 2020-02-06 00:00:00 2020-02-12 23:59:50
2020-03-20 02:50:14 chunk 3 2020-02-06 00:00:00 2020-02-12 23:59:20
2020-03-20 02:50:14 chunk 4 2020-02-06 00:00:04 2020-02-12 23:59:34
2020-03-20 02:50:14 chunk 5 2020-02-06 00:00:00 2020-02-12 23:59:21
2020-03-20 02:50:14 chunk 6 2020-02-06 00:00:19 2020-02-12 23:58:28
2020-03-20 02:50:14 chunk 7 2020-02-06 00:00:07 2020-02-12 23:57:21
2020-03-20 02:50:14 chunk 8 2020-02-06 00:03:24 2020-02-12 23:59:30
2020-03-20 02:50:14 chunk 9 2020-02-06 00:00:04 2020-02-12 23:56:27
2020-03-20 02:50:14 chunk 10 2020-02-06 00:00:00 2020-02-12 23:59:59
2020-03-20 02:50:14 chunk 11 2020-02-06 00:00:07 2020-02-12 23:59:52
2020-03-20 02:50:14 chunk 12 2020-02-06 00:00:00 2020-02-12 23:57:21
2020-03-20 02:50:14 chunk 13 2020-02-06 00:00:00 2020-02-12 23:55:04
2020-03-20 02:50:15 chunk 14 2020-02-06 00:0

2020-03-20 02:50:22 chunk 120 2020-02-06 00:00:00 2020-02-12 23:58:13
2020-03-20 02:50:22 chunk 121 2020-02-06 00:00:36 2020-02-12 23:59:55
2020-03-20 02:50:22 chunk 122 2020-02-06 00:00:00 2020-02-12 23:58:47
2020-03-20 02:50:22 chunk 123 2020-02-06 00:00:00 2020-02-12 23:55:27
2020-03-20 02:50:22 chunk 124 2020-02-06 00:00:00 2020-02-12 23:56:31
2020-03-20 02:50:22 chunk 125 2020-02-06 00:00:00 2020-02-12 23:59:11
2020-03-20 02:50:22 chunk 126 2020-02-06 00:00:16 2020-02-12 23:59:45
2020-03-20 02:50:22 chunk 127 2020-02-06 00:01:18 2020-02-12 23:59:50
2020-03-20 02:50:22 chunk 128 2020-02-06 00:00:00 2020-02-12 23:54:18
2020-03-20 02:50:22 chunk 129 2020-02-06 00:00:19 2020-02-12 23:59:51
2020-03-20 02:50:23 chunk 130 2020-02-06 00:00:00 2020-02-12 23:57:17
2020-03-20 02:50:23 chunk 131 2020-02-06 00:00:00 2020-02-12 23:59:50
2020-03-20 02:50:23 chunk 132 2020-02-06 00:00:04 2020-02-12 23:58:46
2020-03-20 02:50:23 chunk 133 2020-02-06 00:00:48 2020-02-12 23:59:09
2020-03-20 02:50:23 

2020-03-20 02:50:30 chunk 240 2020-02-06 00:00:20 2020-02-12 23:54:55
2020-03-20 02:50:30 chunk 241 2020-02-06 00:00:00 2020-02-12 23:59:13
2020-03-20 02:50:30 chunk 242 2020-02-06 00:00:09 2020-02-12 23:59:59
2020-03-20 02:50:30 chunk 243 2020-02-06 00:00:07 2020-02-12 23:54:38
2020-03-20 02:50:30 chunk 244 2020-02-06 00:00:56 2020-02-12 23:56:57
2020-03-20 02:50:30 chunk 245 2020-02-06 00:00:37 2020-02-12 23:58:36
2020-03-20 02:50:30 chunk 246 2020-02-06 00:00:00 2020-02-12 23:57:42
2020-03-20 02:50:31 chunk 247 2020-02-06 00:00:14 2020-02-12 23:59:55
2020-03-20 02:50:31 chunk 248 2020-02-06 00:00:00 2020-02-12 23:57:58
2020-03-20 02:50:31 chunk 249 2020-02-06 00:00:00 2020-02-12 23:59:01
2020-03-20 02:50:31 chunk 250 2020-02-06 00:00:01 2020-02-12 23:58:49
2020-03-20 02:50:31 chunk 251 2020-02-06 00:00:08 2020-02-12 23:58:11
2020-03-20 02:50:31 chunk 252 2020-02-06 00:00:10 2020-02-12 23:59:21
2020-03-20 02:50:31 chunk 253 2020-02-06 00:00:40 2020-02-12 23:57:40
2020-03-20 02:50:31 

2020-03-20 02:50:38 chunk 360 2020-02-06 00:00:06 2020-02-12 23:57:05
2020-03-20 02:50:38 chunk 361 2020-02-06 00:00:01 2020-02-12 23:59:49
2020-03-20 02:50:38 chunk 362 2020-02-06 00:00:00 2020-02-12 23:59:52
2020-03-20 02:50:39 chunk 363 2020-02-06 00:00:10 2020-02-12 23:57:20
2020-03-20 02:50:39 chunk 364 2020-02-06 00:00:33 2020-02-12 23:54:48
2020-03-20 02:50:39 chunk 365 2020-02-06 00:02:38 2020-02-12 23:59:35
2020-03-20 02:50:39 chunk 366 2020-02-06 00:00:00 2020-02-12 23:55:59
2020-03-20 02:50:39 chunk 367 2020-02-06 00:00:05 2020-02-12 23:57:56
2020-03-20 02:50:39 chunk 368 2020-02-06 00:00:01 2020-02-12 23:56:10
2020-03-20 02:50:39 chunk 369 2020-02-06 00:00:01 2020-02-12 23:56:52
2020-03-20 02:50:39 chunk 370 2020-02-06 00:00:24 2020-02-12 23:59:42
2020-03-20 02:50:39 chunk 371 2020-02-06 00:00:00 2020-02-12 23:59:42
2020-03-20 02:50:39 chunk 372 2020-02-06 00:00:20 2020-02-12 23:59:42
2020-03-20 02:50:39 chunk 373 2020-02-06 00:02:35 2020-02-12 23:59:22
2020-03-20 02:50:39 

2020-03-20 02:50:47 chunk 480 2020-02-06 00:00:00 2020-02-12 23:59:14
2020-03-20 02:50:47 chunk 481 2020-02-06 00:00:16 2020-02-12 23:57:27
2020-03-20 02:50:47 chunk 482 2020-02-06 00:00:00 2020-02-12 23:59:01
2020-03-20 02:50:47 chunk 483 2020-02-06 00:00:32 2020-02-12 23:57:52
2020-03-20 02:50:47 chunk 484 2020-02-06 00:01:03 2020-02-12 23:54:19
2020-03-20 02:50:47 chunk 485 2020-02-06 00:00:20 2020-02-12 23:59:02
2020-03-20 02:50:47 chunk 486 2020-02-06 00:00:00 2020-02-12 23:58:41
2020-03-20 02:50:47 chunk 487 2020-02-06 00:00:00 2020-02-12 23:57:08
2020-03-20 02:50:47 chunk 488 2020-02-06 00:00:36 2020-02-12 23:59:14
2020-03-20 02:50:47 chunk 489 2020-02-06 00:00:01 2020-02-12 23:59:44
2020-03-20 02:50:47 chunk 490 2020-02-06 00:00:00 2020-02-12 23:49:05
2020-03-20 02:50:47 chunk 491 2020-02-06 00:00:04 2020-02-12 23:59:01
2020-03-20 02:50:47 chunk 492 2020-02-06 00:00:34 2020-02-12 23:59:31
2020-03-20 02:50:47 chunk 493 2020-02-06 00:00:00 2020-02-12 23:59:41
2020-03-20 02:50:48 

...

2020-03-20 02:51:03 chunk 720 2020-02-06 00:00:25 2020-02-12 23:56:06
2020-03-20 02:51:03 chunk 721 2020-02-06 00:00:09 2020-02-12 23:58:43
2020-03-20 02:51:03 chunk 722 2020-02-06 00:00:00 2020-02-12 23:59:47
2020-03-20 02:51:03 chunk 723 2020-02-06 00:00:09 2020-02-12 23:59:42
2020-03-20 02:51:03 chunk 724 2020-02-06 00:00:14 2020-02-12 23:57:06
2020-03-20 02:51:03 chunk 725 2020-02-06 00:00:01 2020-02-12 23:59:40
2020-03-20 02:51:03 chunk 726 2020-02-06 00:00:00 2020-02-12 23:59:12
2020-03-20 02:51:04 chunk 727 2020-02-06 00:00:12 2020-02-12 23:57:59
2020-03-20 02:51:04 chunk 728 2020-02-06 00:00:16 2020-02-12 23:58:24
2020-03-20 02:51:04 chunk 729 2020-02-06 00:00:00 2020-02-12 23:58:14
2020-03-20 02:51:04 chunk 730 2020-02-06 00:00:02 2020-02-12 23:54:47
2020-03-20 02:51:04 chunk 731 2020-02-06 00:00:13 2020-02-12 23:59:30
2020-03-20 02:51:04 chunk 732 2020-02-06 00:00:00 2020-02-12 23:58:36
2020-03-20 02:51:04 chunk 733 2020-02-06 00:00:00 2020-02-12 23:58:28
2020-03-20 02:51:04 

2020-03-20 02:51:11 chunk 840 2020-02-06 00:01:26 2020-02-12 23:59:01
2020-03-20 02:51:11 chunk 841 2020-02-06 00:03:01 2020-02-12 23:57:56
2020-03-20 02:51:12 chunk 842 2020-02-06 00:00:01 2020-02-12 23:59:15
2020-03-20 02:51:12 chunk 843 2020-02-06 00:00:00 2020-02-12 23:56:39
2020-03-20 02:51:12 chunk 844 2020-02-06 00:00:19 2020-02-12 23:59:36
2020-03-20 02:51:12 chunk 845 2020-02-06 00:00:09 2020-02-12 23:58:53
2020-03-20 02:51:12 chunk 846 2020-02-06 00:00:46 2020-02-12 23:59:28
2020-03-20 02:51:12 chunk 847 2020-02-06 00:04:27 2020-02-12 23:58:49
2020-03-20 02:51:12 chunk 848 2020-02-06 00:00:04 2020-02-12 23:58:41
2020-03-20 02:51:12 chunk 849 2020-02-06 00:00:00 2020-02-12 23:59:59
2020-03-20 02:51:12 chunk 850 2020-02-06 00:02:38 2020-02-12 23:56:15
2020-03-20 02:51:12 chunk 851 2020-02-06 00:00:06 2020-02-12 23:56:40
2020-03-20 02:51:12 chunk 852 2020-02-06 00:00:00 2020-02-12 23:57:54
2020-03-20 02:51:12 chunk 853 2020-02-06 00:00:21 2020-02-12 23:55:56
2020-03-20 02:51:12 

2020-03-20 02:51:21 chunk 959 2020-02-06 00:01:16 2020-02-12 23:58:42
2020-03-20 02:51:21 chunk 960 2020-02-06 00:00:48 2020-02-12 23:59:50
2020-03-20 02:51:21 chunk 961 2020-02-06 00:02:25 2020-02-12 23:59:36
2020-03-20 02:51:21 chunk 962 2020-02-06 00:00:08 2020-02-12 23:58:50
2020-03-20 02:51:22 chunk 963 2020-02-06 00:00:25 2020-02-12 23:54:49
2020-03-20 02:51:22 chunk 964 2020-02-06 00:00:00 2020-02-12 23:58:24
2020-03-20 02:51:22 chunk 965 2020-02-06 00:00:00 2020-02-12 23:59:59
2020-03-20 02:51:22 chunk 966 2020-02-06 00:00:03 2020-02-12 23:59:24
2020-03-20 02:51:22 chunk 967 2020-02-06 00:01:41 2020-02-12 23:54:37
2020-03-20 02:51:22 chunk 968 2020-02-06 00:01:04 2020-02-12 23:59:54
2020-03-20 02:51:22 chunk 969 2020-02-06 00:00:31 2020-02-12 23:59:42
2020-03-20 02:51:23 chunk 970 2020-02-06 00:00:12 2020-02-12 23:56:14
2020-03-20 02:51:23 chunk 971 2020-02-06 00:00:59 2020-02-12 23:52:15
2020-03-20 02:51:23 chunk 972 2020-02-06 00:00:44 2020-02-12 23:57:46
2020-03-20 02:51:23 

2020-03-20 02:51:33 chunk 1075 2020-02-06 00:00:06 2020-02-12 23:59:22
2020-03-20 02:51:33 chunk 1076 2020-02-06 00:00:13 2020-02-12 23:59:55
2020-03-20 02:51:33 chunk 1077 2020-02-06 00:00:00 2020-02-12 23:59:37
2020-03-20 02:51:33 chunk 1078 2020-02-06 00:00:00 2020-02-12 23:55:38
2020-03-20 02:51:33 chunk 1079 2020-02-06 00:00:00 2020-02-12 23:59:26
2020-03-20 02:51:33 chunk 1080 2020-02-06 00:00:00 2020-02-12 23:59:30
2020-03-20 02:51:33 chunk 1081 2020-02-06 00:00:00 2020-02-12 23:59:59
2020-03-20 02:51:33 chunk 1082 2020-02-06 00:00:39 2020-02-12 23:58:52
2020-03-20 02:51:34 chunk 1083 2020-02-06 00:00:10 2020-02-12 23:56:33
2020-03-20 02:51:34 chunk 1084 2020-02-06 00:00:59 2020-02-12 23:56:29
2020-03-20 02:51:34 chunk 1085 2020-02-06 00:00:54 2020-02-12 23:59:27
2020-03-20 02:51:34 chunk 1086 2020-02-06 00:00:24 2020-02-12 23:58:09
2020-03-20 02:51:34 chunk 1087 2020-02-06 00:00:02 2020-02-12 23:59:15
2020-03-20 02:51:34 chunk 1088 2020-02-06 00:00:00 2020-02-12 23:59:38
2020-0

KeyboardInterrupt: 

In [ ]:
chnks_val = pd.read_csv(f'{p_in}/val_{valtmstmp}.tsv',sep='\x01',
                    header=None,names=cols, 
                        chunksize=1e4
                       )
for ichnk,df in enumerate(chnks_val):
    mintm,maxtm=df.tm.min(),df.tm.max()
    mintm,maxtm=(datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (mintm,maxtm))
#     display(df.head(1))
    print(dtnow(), 'chunk', ichnk, mintm,maxtm)
ichnk

2020-03-20 02:51:48 chunk 0 2020-02-13 00:00:00 2020-02-19 23:59:24
2020-03-20 02:51:48 chunk 1 2020-02-13 00:00:05 2020-02-19 23:58:27
2020-03-20 02:51:48 chunk 2 2020-02-13 00:00:09 2020-02-19 23:56:53
2020-03-20 02:51:48 chunk 3 2020-02-13 00:00:47 2020-02-19 23:58:27
2020-03-20 02:51:48 chunk 4 2020-02-13 00:00:00 2020-02-19 23:59:11
2020-03-20 02:51:48 chunk 5 2020-02-13 00:00:00 2020-02-19 23:59:53
2020-03-20 02:51:48 chunk 6 2020-02-13 00:00:40 2020-02-19 23:57:11
2020-03-20 02:51:48 chunk 7 2020-02-13 00:00:23 2020-02-19 23:57:31
2020-03-20 02:51:49 chunk 8 2020-02-13 00:00:08 2020-02-19 23:59:50
2020-03-20 02:51:49 chunk 9 2020-02-13 00:00:40 2020-02-19 23:59:14
2020-03-20 02:51:49 chunk 10 2020-02-13 00:00:00 2020-02-19 23:58:57
2020-03-20 02:51:49 chunk 11 2020-02-13 00:00:03 2020-02-19 23:59:01
2020-03-20 02:51:49 chunk 12 2020-02-13 00:00:00 2020-02-19 23:59:16
2020-03-20 02:51:49 chunk 13 2020-02-13 00:02:25 2020-02-19 23:59:46
2020-03-20 02:51:49 chunk 14 2020-02-13 00:0

2020-03-20 02:51:59 chunk 120 2020-02-13 00:00:00 2020-02-19 23:57:50
2020-03-20 02:51:59 chunk 121 2020-02-13 00:00:00 2020-02-19 23:59:50
2020-03-20 02:51:59 chunk 122 2020-02-13 00:00:00 2020-02-19 23:52:18
2020-03-20 02:51:59 chunk 123 2020-02-13 00:00:00 2020-02-19 23:52:56
2020-03-20 02:51:59 chunk 124 2020-02-13 00:00:00 2020-02-19 23:59:02
2020-03-20 02:51:59 chunk 125 2020-02-13 00:00:00 2020-02-19 23:59:27
2020-03-20 02:51:59 chunk 126 2020-02-13 00:00:47 2020-02-19 23:59:13
2020-03-20 02:51:59 chunk 127 2020-02-13 00:00:00 2020-02-19 23:59:06
2020-03-20 02:52:00 chunk 128 2020-02-13 00:00:00 2020-02-19 23:57:25
2020-03-20 02:52:00 chunk 129 2020-02-13 00:00:30 2020-02-19 23:59:09
2020-03-20 02:52:00 chunk 130 2020-02-13 00:01:39 2020-02-19 23:57:18
2020-03-20 02:52:00 chunk 131 2020-02-13 00:00:00 2020-02-19 23:58:01
2020-03-20 02:52:00 chunk 132 2020-02-13 00:00:02 2020-02-19 23:58:29
2020-03-20 02:52:00 chunk 133 2020-02-13 00:03:40 2020-02-19 23:59:32
2020-03-20 02:52:00 

2020-03-20 02:52:09 chunk 238 2020-02-13 00:00:05 2020-02-19 23:55:31
2020-03-20 02:52:09 chunk 239 2020-02-13 00:00:00 2020-02-19 23:55:25
2020-03-20 02:52:09 chunk 240 2020-02-13 00:00:00 2020-02-19 23:56:32
2020-03-20 02:52:09 chunk 241 2020-02-13 00:00:00 2020-02-19 23:58:32
2020-03-20 02:52:09 chunk 242 2020-02-13 00:00:40 2020-02-19 23:56:42
2020-03-20 02:52:09 chunk 243 2020-02-13 00:00:47 2020-02-19 23:57:30
2020-03-20 02:52:09 chunk 244 2020-02-13 00:00:00 2020-02-19 23:59:59
2020-03-20 02:52:09 chunk 245 2020-02-13 00:00:00 2020-02-19 23:57:55
2020-03-20 02:52:09 chunk 246 2020-02-13 00:00:00 2020-02-19 23:59:09
2020-03-20 02:52:10 chunk 247 2020-02-13 00:00:00 2020-02-19 23:59:07
2020-03-20 02:52:10 chunk 248 2020-02-13 00:00:00 2020-02-19 23:57:34
2020-03-20 02:52:10 chunk 249 2020-02-13 00:00:00 2020-02-19 23:55:26
2020-03-20 02:52:10 chunk 250 2020-02-13 00:00:00 2020-02-19 23:59:04
2020-03-20 02:52:10 chunk 251 2020-02-13 00:00:00 2020-02-19 23:58:24
2020-03-20 02:52:10 

2020-03-20 02:52:21 chunk 357 2020-02-13 00:00:00 2020-02-19 23:56:00
2020-03-20 02:52:21 chunk 358 2020-02-13 00:00:05 2020-02-19 23:55:26
2020-03-20 02:52:21 chunk 359 2020-02-13 00:01:03 2020-02-19 23:59:52
2020-03-20 02:52:21 chunk 360 2020-02-13 00:00:00 2020-02-19 23:56:10
2020-03-20 02:52:21 chunk 361 2020-02-13 00:00:04 2020-02-19 23:59:49
2020-03-20 02:52:21 chunk 362 2020-02-13 00:00:00 2020-02-19 23:59:35
2020-03-20 02:52:21 chunk 363 2020-02-13 00:00:00 2020-02-19 23:57:08
2020-03-20 02:52:21 chunk 364 2020-02-13 00:00:00 2020-02-19 23:58:03
2020-03-20 02:52:21 chunk 365 2020-02-13 00:00:15 2020-02-19 23:58:58
2020-03-20 02:52:21 chunk 366 2020-02-13 00:00:16 2020-02-19 23:58:47
2020-03-20 02:52:22 chunk 367 2020-02-13 00:00:00 2020-02-19 23:56:54
2020-03-20 02:52:22 chunk 368 2020-02-13 00:00:00 2020-02-19 23:56:00
2020-03-20 02:52:22 chunk 369 2020-02-13 00:00:00 2020-02-19 23:51:23
2020-03-20 02:52:22 chunk 370 2020-02-13 00:00:18 2020-02-19 23:59:59
2020-03-20 02:52:22 

2020-03-20 02:52:29 chunk 477 2020-02-13 00:00:28 2020-02-19 23:59:15
2020-03-20 02:52:29 chunk 478 2020-02-13 00:00:00 2020-02-19 23:58:38
2020-03-20 02:52:29 chunk 479 2020-02-13 00:02:04 2020-02-19 23:58:49
2020-03-20 02:52:29 chunk 480 2020-02-13 00:00:00 2020-02-19 23:57:45
2020-03-20 02:52:29 chunk 481 2020-02-13 00:01:04 2020-02-19 23:54:35
2020-03-20 02:52:30 chunk 482 2020-02-13 00:00:32 2020-02-19 23:59:10
2020-03-20 02:52:30 chunk 483 2020-02-13 00:00:00 2020-02-19 23:58:12
2020-03-20 02:52:30 chunk 484 2020-02-13 00:00:50 2020-02-19 23:59:11
2020-03-20 02:52:30 chunk 485 2020-02-13 00:00:54 2020-02-19 23:59:52
2020-03-20 02:52:30 chunk 486 2020-02-13 00:00:00 2020-02-19 23:58:47
2020-03-20 02:52:30 chunk 487 2020-02-13 00:00:05 2020-02-19 23:59:20
2020-03-20 02:52:30 chunk 488 2020-02-13 00:00:07 2020-02-19 23:59:17
2020-03-20 02:52:30 chunk 489 2020-02-13 00:00:00 2020-02-19 23:59:10
2020-03-20 02:52:30 chunk 490 2020-02-13 00:00:00 2020-02-19 23:58:33
2020-03-20 02:52:30 